In [1]:
import sys
import random
import os
from pathlib import Path
import shutil
import json

import argparse
from tqdm import tqdm
import spacy
from spacy.gold import minibatch
from spacy.language import Language
from spacy import util

import jsonlines 
from tqdm import tqdm
from bs4 import BeautifulSoup

import exrex
from string import punctuation

In [2]:
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

class EntityMatcher(object):
#     name = 'entity_matcher'

    def __init__(self, nlp, terms, label):
#         patterns = [nlp(term) for term in tqdm(terms)]
        patterns = [nlp.make_doc(term) for term in tqdm(terms)]
        
        if label =='OG':
            self.matcher = PhraseMatcher(nlp.vocab, attr="LOWER") # attribute lower is removed. e.g., IF A gene ==> if a ..
        else:
            self.matcher = PhraseMatcher(nlp.vocab) 
        
        self.matcher.add(label, None, *patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        seen_tokens = set()
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
            # check for end - 1 here because boundaries are inclusive
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc
    
#         matches = set(
#                     [(m_id, start, end) for m_id, start, end in matches if start != end]
#                 )
#         get_sort_key = lambda m: (m[2] - m[1], m[1])
#         matches = sorted(matches, key=get_sort_key, reverse=True)
#         entities = list(doc.ents)
#         new_entities = []
#         seen_tokens = set()
#         for match_id, start, end in matches:
#             if any(t.ent_type for t in doc[start:end]):
#                 continue
#             # check for end - 1 here because boundaries are inclusive
#             if start not in seen_tokens and end - 1 not in seen_tokens:
#                 new_entities.append(Span(doc, start, end, label=match_id))
#                 entities = [
#                     e for e in entities if not (e.start < end and e.end > start)
#                 ]
#                 seen_tokens.update(range(start, end))
#         doc.ents = entities + new_entities
#         return doc     

In [3]:
Diseases = '/nfs/gns/literature/lit-textmining-pipelines/automata/DiseaseDictionary.mwt' #(# of terms - 59088)
Genes_Proteins = '/nfs/gns/literature/lit-textmining-pipelines/automata/swissprot_Sept2014.2.3.mwt' #(# of terms - 347929)
Organism = '/nfs/gns/literature/lit-textmining-pipelines/automata/Organisms150507.2.mwt' #(# of terms - 1683021)

gene_patterns = []
disease_patterns = []
organisms_patterns = []


print('Loading Genes Dictionary')
with open(Genes_Proteins, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    pattern_types = soup.find_all('t')
    for pattern in tqdm(pattern_types):
        gene_patterns.append(pattern.text)
#     pattern_types = soup.find_all('r')
#     for pattern in tqdm(pattern_types):
#         gene_patterns.append(pattern.text)    
        
        

print('loading Disease Dictionary')
with open(Diseases, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    pattern_types = soup.find_all('t')
    for pattern in tqdm(pattern_types):
        disease_patterns.append(pattern.text)


        
# Organism = '/nfs/gns/literature/machine-learning/Dictionaries/MWT/organisms_toy.mwt' #(# of terms - 1683021)        
# organisms_patterns = []
print('Loading Organims Dictionary')
with open(Organism, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')
    pattern_types = soup.find_all('r')
for pattern in tqdm(pattern_types):
    dictionary_term = exrex.getone(pattern.text.replace('[ \\-_]*', ' ').replace('[^A-Za-z0-9]', ''))
    clean_term = ' '.join(dictionary_term.split())
    organisms_patterns.append(clean_term)
  

Loading Genes Dictionary


100%|██████████| 347929/347929 [00:01<00:00, 254233.12it/s]


loading Disease Dictionary


100%|██████████| 59088/59088 [00:00<00:00, 262426.85it/s]


Loading Organims Dictionary


100%|██████████| 1683023/1683023 [03:03<00:00, 9168.87it/s]


In [4]:
## load the blank model

# nlp = spacy.blank('en')



In [5]:
best_model_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-lg/best'
nlp = spacy.load(best_model_path)
print("Loaded from", best_model_path)

Loaded from /nfs/gns/literature/Santosh_Tirunagari/GitHub/spacy_models/pubmed-pmc-lg/best


In [6]:
entity_matcher_GP = EntityMatcher(nlp, gene_patterns, 'GP')
entity_matcher_DS = EntityMatcher(nlp, disease_patterns, 'DS')
entity_matcher_OG = EntityMatcher(nlp, organisms_patterns, 'OG')

100%|██████████| 1683023/1683023 [03:04<00:00, 9131.11it/s]


In [1]:
entity_matcher_GP.name = 'GP_dict'
entity_matcher_DS.name = 'DS_dict'
entity_matcher_OG.name = 'OG_dict'

# nlp.add_pipe(entity_matcher_GP)
# nlp.add_pipe(entity_matcher_OG, after='GP_dict')
# nlp.add_pipe(entity_matcher_DS, after='OG_dict')


nlp.add_pipe(entity_matcher_GP, after='ner')
nlp.add_pipe(entity_matcher_DS, after='ner')
nlp.add_pipe(entity_matcher_OG, after='GP_dict')

NameError: name 'entity_matcher_GP' is not defined

In [8]:
# nlp.add_pipe(nlp2.pipeline[-1][-1], 'ner')

# nlp.pipe_names

In [9]:
# nlp.remove_pipe('DS_dict')
# nlp.remove_pipe('GP_dict')
# nlp.remove_pipe('OG_dict')
# nlp.add_pipe(entity_matcher_GP, after='ner')
# nlp.add_pipe(entity_matcher_DS, after='ner')
# nlp.remove_pipe('GP_dict')
# print(nlp.pipe_names)
# nlp.add_pipe(entity_matcher_GP, after='ner')
# nlp.add_pipe(entity_matcher_DS, before='ner')
print(nlp.pipe_names)

['ner', 'DS_dict', 'GP_dict', 'OG_dict']


In [10]:
text = 'Interspecific tumor ZBTB12 PT Sergef Ribonuclease 5 EN116 overgrowth TGN48 Uncharacterized protein Rv2292c phenocopies the s.Crocodylurus Transmembrane protein 2 tarahumara frogs Putative HLA class I histocompatibility antigen, alpha chain H female gametophytic mutants feronia/sirène (fer/srn), lorelei (lre), nortia (nta), Turan (tun), evAn (evn), and Zea mays embryo sac 4 (ZmES4) RNAi-lines1314151617181920, PtdInsTP β,  hmg2l1, MXL8.8 which are defective in the reception of intraspecific PTs. GTP'
# text = 'These ractory cytopenia with unilineage dysplasia iris neoplasms results indicate the dentate gyrus is breast tumor mostly comprised of mature neurons (NeuN), along with a smaller population of precursor cells (nestin) and newly differentiated neurons (DCX), which corresponds to prior findings examining the relative number of each cell population in the dentate gyrus, indicating the proportion of cells labeled by the sensor approximately reflects physiological proportions [21, 22]. '
# text = ' However, if a pollen grain originating from a different species (interspecific pollination) is placed on a plant\'s stigma, all the communication processes desc     27 ribed above have the potential to act as pre-zygotic post-pollination barriers.'
# text = 'Bradyrhizobium sp. B1P5L4 STM2336, M. zalophi, Tarahumara Frogs, Residual erythrocytes were removed by hypotonic lysis'
sentence =nlp(text)

print(sentence)
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~ **** ~~~~~~~~~~~~~~~~~~~~~~~')
for ent in sentence.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Interspecific tumor ZBTB12 PT Sergef Ribonuclease 5 EN116 overgrowth TGN48 Uncharacterized protein Rv2292c phenocopies the s.Crocodylurus Transmembrane protein 2 tarahumara frogs Putative HLA class I histocompatibility antigen, alpha chain H female gametophytic mutants feronia/sirène (fer/srn), lorelei (lre), nortia (nta), Turan (tun), evAn (evn), and Zea mays embryo sac 4 (ZmES4) RNAi-lines1314151617181920, PtdInsTP β,  hmg2l1, MXL8.8 which are defective in the reception of intraspecific PTs. GTP
~~~~~~~~~~~~~~~~~~~~~~~~~~~ **** ~~~~~~~~~~~~~~~~~~~~~~~
tumor 14 19 DS
ZBTB12 20 26 GP
Sergef 30 36 GP
Ribonuclease 37 49 GP
EN116 52 57 GP
TGN48 69 74 GP
Uncharacterized protein Rv2292c 75 106 GP
Transmembrane protein 2 138 161 GP
frogs 173 178 OG
Putative HLA class I histocompatibility antigen, alpha chain H 179 241 GP
feronia 270 277 GP
sirène 278 284 GP
srn 290 293 GP
Zea mays 354 362 OG
PtdInsTP β 412 422 GP
hmg2l1 425 431 GP
MXL8.8 433 439 GP


In [11]:
from nltk.tokenize import WordPunctTokenizer, wordpunct_tokenize


def convert2IOB(text_data, ner_tags):
    tokenizer = WordPunctTokenizer()

    tokens = []
    ners = []
    spans = []

    split_text = tokenizer.tokenize(text_data)
    span_text = list(tokenizer.span_tokenize(text_data))
    # for each word token append 'O'
    arr = ['O'] * len(split_text)

    if ner_tags:
        try:
            ner_tags = literal_eval(ner_tags)
        except:
            pass

    elif isinstance(ner_tags, float) or ner_tags is None:
        return zip(split_text, arr)
    else:
        return zip(split_text, arr)

    for each_tag in ner_tags:
        span_list = (each_tag[0], each_tag[1])
        token_list = wordpunct_tokenize(each_tag[2])
        ner_list = wordpunct_tokenize(each_tag[3])

        if (len(token_list) > len(ner_list)):
            ner_list = len(token_list) * ner_list
        for i in range(0, len(ner_list)):
            # The logic here is look for the first B-tag and then append I-tag next
            if (i == 0):
                ner_list[i] = 'B-' + ner_list[i]
            else:
                ner_list[i] = 'I-' + ner_list[i]

        tokens.append(token_list)
        ners.append(ner_list)
        spans.append(span_list)

    split_token_span_list = list(zip(split_text, span_text))
    span_ner_list = list(zip(spans, ners))

    sub_spans = []  # get sub spans from the full spans of the ner

    for each_span_ner_list in span_ner_list:
        # in full range ner e.g., [144, 150, 'GM-CSF', 'GP']
        count = 0
        # count is to keep track of the B, I, sub tags in the ner list
        for each_token in split_token_span_list:
            sub_spans_ = find_sub_span(each_token[1], each_span_ner_list[0])
            if sub_spans_:
                sub_spans.append([sub_spans_, each_span_ner_list[1][count]])
                count = count + 1

    for i, each_span_token in enumerate(split_token_span_list):
        for each_ner_span in sub_spans:
            if each_span_token[1] == each_ner_span[0]:
                arr[i] = ''.join(each_ner_span[1])

    return zip(split_text, arr)


In [12]:
import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import csv


def find_sub_span(sub_span_range, full_spans_range):
    # if a sub span is present in full span return it
    if sub_span_range[0] in range(full_spans_range[0], full_spans_range[1]):
        return sub_span_range
    
    

test_set = '/nfs/gns/literature/machine-learning/evaluation/300articles/CSV formats/test.csv'

result_path = '/nfs/gns/literature/machine-learning/evaluation/300articles/ML-NER/Dictionary_fused_en_pubmed_pmc_lg/'


df_45 = pd.read_csv(test_set, sep = '\t', names = ['pmcid', 'sentence','ner'])

with open(result_path + 'Dictionary_fused_en_pubmed_pmc_lg_iob.csv', 'w', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index, row in tqdm(df_45.iterrows(), total=df_45.shape[0]):
#         print(row['ner'])
        text = row['sentence'] # .encode('utf-8').decode('utf-8')
#         print(text)
        sentence = nlp(text)
        ml_ner =[]
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)

        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')


100%|██████████| 17957/17957 [01:19<00:00, 225.89it/s]


In [ ]:
## Test best model performance on 2000 set

from ast import literal_eval

epmc_annotations_2000 = '/nfs/gns/literature/machine-learning/evaluation/2000articles/europePMC-NER/annotations_API/full_sentences/tagged_sentences/Europe_PMC_annotation.csv'

result_path = '/nfs/gns/literature/machine-learning/evaluation/2000articles/ML-NER/Dictionary_fused_en_pubmed_pmc_lg/'


df_2000 = pd.read_csv(epmc_annotations_2000, sep = '\t', names = ['pmcid', 'section', 'sentence','ner'])


with open(result_path + 'Dictionary_en-pubmed-pmc-lg_2000_iob.csv', 'a', newline='\n') as f1:
    ml_writer = csv.writer(f1, delimiter='\t', lineterminator='\n')
    
    for index, row in tqdm(df_2000.iterrows(), total=df_2000.shape[0]):
#         print(row['ner'])
        text = row['sentence'].encode('utf-8').decode('utf-8')
#         print(text)
        sentence = nlp(text)
        ml_ner =[]
        for ent in sentence.ents:
            ml_ner.append([ent.start_char, ent.end_char, ent.text, ent.label_])

        tagged_tokens = convert2IOB(text, ml_ner)

        for each_word in tagged_tokens:
            ml_writer.writerow(list(each_word))
        ml_writer.writerow('')
        
        
